In [0]:
%run "../includes/configuration"

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
df_races = spark.read.format("delta").load(f"{processed_folder_path}/races").withColumnRenamed("name", "race_name").withColumnRenamed("race_timestamp", "race_date")

In [0]:
df_circuits = spark.read.format("delta").load(f"{processed_folder_path}/circuits").withColumnRenamed("name", "circuit_name")

In [0]:
df_drivers = spark.read.format("delta").load(f"{processed_folder_path}/drivers")

In [0]:
df_results = spark.read.format("delta").load(f"{processed_folder_path}/results").filter(f"file_date = '{v_file_date}'")

In [0]:
df_constructors = spark.read.format("delta").load(f"{processed_folder_path}/constructors") \
    .select(col("constructor_id").alias("c_id"), col("name").alias("team"))

In [0]:
df_joined_races_circuits = df_races.join(df_circuits, df_races.circuit_id == df_circuits.circuit_id, "inner") \
    .select(df_races.race_id,df_races.race_year, df_races.race_name, df_races.race_date, df_circuits.location)

In [0]:
df_joined_drivers_results = df_results.join(df_drivers, df_results.driver_id == df_drivers.driver_id, "inner") \
    .select(df_results.race_id, df_results.driver_id, df_results.constructor_id, df_results.grid, df_results.fastest_lap, df_results.time, df_results.points, df_drivers.name, df_drivers.number, df_drivers.nationality, df_results.position)

In [0]:
df_joined_drivers_results = df_joined_drivers_results\
    .withColumnRenamed("name", "driver_name")\
    .withColumnRenamed("number", "driver_number")\
    .withColumnRenamed("nationality", "driver_nationality")\
    .withColumnRenamed("time", "race_time")

In [0]:
df_drivers_results_constructors = df_joined_drivers_results\
    .join(df_constructors, df_joined_drivers_results.constructor_id == df_constructors.c_id, "inner")\
    .select([*df_joined_drivers_results.columns, 'team'])

In [0]:
df_final = df_joined_races_circuits.join(df_drivers_results_constructors, df_joined_races_circuits.race_id == df_drivers_results_constructors.race_id, "inner") \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("file_date", lit(v_file_date)) \
    .drop(df_joined_races_circuits.race_id)

In [0]:
%run "../includes/common_functions"

In [0]:
# apply_incremental_load(df_final, "f1_presentation.race_results", "race_id")


merge_delta_data(df_final, 'f1_presentation','race_results',presentation_folder_path, 'tgt.driver_name = src.driver_name AND tgt.race_id = src.race_id','race_id')